In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-reviews/amazon_review_polarity_csv.tgz
/kaggle/input/amazon-reviews/train.csv
/kaggle/input/amazon-reviews/test.csv


In [3]:
!pip install nltk -q
!pip install textblob -q
!pip install wordcloud -q
!pip install transformers datasets -q
!pip uninstall -y tensorflow tensorflow-text tf-keras tensorflow-decision-forests
!pip install tensorflow==2.15.0 tensorflow-text==2.15.0 tf-keras==2.15.1 tensorflow-decision-forests==1.8.1
!pip install transformers -q
!pip install pyarrow
!pip install torch

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
Found existing installation: tensorflow-text 2.15.0
Uninstalling tensorflow-text-2.15.0:
  Successfully uninstalled tensorflow-text-2.15.0
Found existing installation: tf_keras 2.15.1
Uninstalling tf_keras-2.15.1:
  Successfully uninstalled tf_keras-2.15.1
Found existing installation: tensorflow-decision-forests 1.8.1
Uninstalling tensorflow-decision-forests-1.8.1:
  Successfully uninstalled tensorflow-decision-forests-1.8.1
  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached tensorflow_text-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached tf_keras-2.15.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_decision_forests-1.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
Using cached tensorflow-2.15.0

In [4]:
import numpy as np
import pandas as pd
import tarfile
import os
import matplotlib.pyplot as plt
import string
import nltk
import re
import seaborn as sns
import tensorflow as tf
import transformers
import torch



from datasets import load_dataset
from warnings import filterwarnings
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import classification_report,accuracy_score,precision_recall_fscore_support
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from textblob import Word, TextBlob
from wordcloud import WordCloud
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from sklearn.preprocessing import LabelEncoder


In [5]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
train = pd.read_csv("/kaggle/input/amazon-reviews/train.csv")
test = pd.read_csv("/kaggle/input/amazon-reviews/test.csv")

In [7]:
train

,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
0,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
1,2,Amazing!,This soundtrack is my favorite music of all ti...
2,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
3,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
4,2,an absolute masterpiece,I am quite sure any of you actually taking the...
...,...,...,...
3599994,1,Don't do it!!,The high chair looks great when it first comes...
3599995,1,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
3599996,1,"compact, but hard to clean","We have a small house, and really wanted two o..."
3599997,1,what is it saying?,not sure what this book is supposed to be. It ...


In [8]:
train.columns = ["polarity","title","text"]
test.columns = ["polarity","title","text"]

In [9]:
train

,polarity,title,text
0,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
1,2,Amazing!,This soundtrack is my favorite music of all ti...
2,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
3,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
4,2,an absolute masterpiece,I am quite sure any of you actually taking the...
...,...,...,...
3599994,1,Don't do it!!,The high chair looks great when it first comes...
3599995,1,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
3599996,1,"compact, but hard to clean","We have a small house, and really wanted two o..."
3599997,1,what is it saying?,not sure what this book is supposed to be. It ...


In [10]:
print("train:")
print(train.isnull().sum())
print("test:")
print(test.isnull().sum())

train:
polarity      0
title       207
text          0
dtype: int64
test:
polarity     0
title       24
text         0
dtype: int64


In [11]:
train['title']=train['title'].fillna('no title')
test['title']=test['title'].fillna('no title')

In [12]:
print("train:")
print(train.isnull().sum())
print("test:")
print(test.isnull().sum())

train:
polarity    0
title       0
text        0
dtype: int64
test:
polarity    0
title       0
text        0
dtype: int64


In [13]:
train = train.dropna(subset=['polarity'])
test = test.dropna(subset=['polarity'])

In [14]:
for col in ['title', 'text']:
    train[col] = train[col].str.lower()

for col in ['title', 'text']:
    test[col] = test[col].str.lower()

In [15]:
# Function to sample an equal number of rows for each polarity

def sample_equal_polarity(df, num_samples):
    unique_polarities = df['polarity'].unique()
    sampled_dfs = []

    for polarity in unique_polarities:
        polarity_df = df[df['polarity'] == polarity]
        if len(polarity_df) > num_samples:
            sampled_dfs.append(polarity_df.sample(num_samples, random_state=1))
        else:
            sampled_dfs.append(polarity_df)

    sampled_df = pd.concat(sampled_dfs)
    return sampled_df

num_samples_per_polarity = 200

train = sample_equal_polarity(train, num_samples_per_polarity)
test = sample_equal_polarity(test, num_samples_per_polarity)

print("Train sample polarity distribution:\n", train['polarity'].value_counts())
print("Test sample polarity distribution:\n", test['polarity'].value_counts())

Train sample polarity distribution:
 polarity
2    200
1    200
Name: count, dtype: int64
Test sample polarity distribution:
 polarity
2    200
1    200
Name: count, dtype: int64


In [16]:
def clean_data(text):
    # remove special characters
    text = text.translate(str.maketrans("", "", string.punctuation))
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # escaped quotes and new lines
    text = text.replace('\\n', ' ').replace('\\t', ' ')
    text = text.replace('\n', ' ').replace('\r', ' ')
    # strip leading and trailing spaces
    text = text.strip()
    return text



for col in ['title', 'text']:
    train[col] = train[col].apply(clean_data)

for col in ['title', 'text']:
    test[col] = test[col].apply(clean_data)

In [17]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [18]:
def preprocess_text(text, method='lemmatization'):
    if pd.isna(text):
        return ""
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word.lower() not in stop_words]  # Remove stopwords
    if method == 'stemming':
        tokens = [stemmer.stem(word) for word in tokens]
    elif method == 'lemmatization':
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

def preprocess_combined(row, method='lemmatization'):
    combined_text = str(row['title']) + ' ' + str(row['text'])
    return preprocess_text(combined_text, method)

train['processed_text'] = train.apply(lambda row: preprocess_combined(row, method='lemmatization'), axis=1)
test['processed_text'] = test.apply(lambda row: preprocess_combined(row, method='lemmatization'), axis=1)

In [19]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['processed_text'])
X_test = vectorizer.transform(test['processed_text'])

In [20]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 13158 stored elements and shape (400, 4924)>
  Coords	Values
  (0, 4110)	0.2202859162330154
  (0, 3403)	0.2202859162330154
  (0, 4412)	0.2202859162330154
  (0, 3215)	0.1684855705153255
  (0, 343)	0.20953637325879063
  (0, 3126)	0.2354365461176356
  (0, 1314)	0.2354365461176356
  (0, 818)	0.18363620039994563
  (0, 4174)	0.4708730922352712
  (0, 2438)	0.4405718324660308
  (0, 798)	0.4708730922352712
  (1, 2009)	0.22188324062313167
  (1, 4655)	0.22686636944971905
  (1, 1910)	0.11298022243904429
  (1, 2606)	0.1967866575477702
  (1, 2424)	0.1967866575477702
  (1, 192)	0.25466631571488196
  (1, 2912)	0.17318638140470927
  (1, 3453)	0.29800318371546874
  (1, 354)	0.23243703353196818
  (1, 806)	0.25466631571488196
  (1, 1113)	0.22188324062313167
  (1, 1931)	0.27882631412846903
  (1, 3424)	0.19408329435796876
  (1, 2095)	0.24604323903671876
  :	:
  (399, 4868)	0.1304347390457384
  (399, 4465)	0.14463726772510888
  (399, 3011)	0.13336

In [21]:
train

,polarity,title,text,processed_text
113684,2,chopat knee strap,the chopat knee strap does what it claims to d...,chopat knee strap chopat knee strap claim doho...
2162445,2,keyboards are a hit,purchased 2 keyboards for grand daughters for ...,keyboard hit purchased 2 keyboard grand daught...
3462086,2,works great,i was given a dvd player that was not compatib...,work great given dvd player compatible tv done...
452360,2,always ivan lins,im from rio de janeiro brasil and i think that...,always ivan lin im rio de janeiro brasil think...
2530821,2,extraordinarily good value,this is a very nice mans watch particularly at...,extraordinarily good value nice man watch part...
...,...,...,...,...
1998226,1,worst movie of the year,this movie moves extremely slow and the story ...,worst movie year movie move extremely slow sto...
490031,1,rubbish,this album is the worst music i have ever hear...,rubbish album worst music ever heardits hardly...
3080138,1,not for me,very strange book just when im getting interes...,strange book im getting interested plot seems ...
1043123,1,disappointed,i wasnt expecting much but was still disappoin...,disappointed wasnt expecting much still disapp...


In [22]:
def convert_polarity_to_sentiment(polarity):
    return 'negative' if polarity == 1 else 'positive'


train['sentiment'] = train['polarity'].apply(convert_polarity_to_sentiment)
test['sentiment'] = test['polarity'].apply(convert_polarity_to_sentiment)

In [23]:
y_train = train['sentiment']
y_test = test['sentiment']

BERT

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(train['processed_text']), padding='max_length', truncation=True, max_length=256, return_tensors='pt')
test_encodings = tokenizer(list(test['processed_text']), padding='max_length', truncation=True, max_length=256, return_tensors='pt')

train_labels = [label - 1 for label in train['polarity']]
test_labels = [label - 1 for label in test['polarity']]

print(set(train_labels))
print(set(test_labels))

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{0, 1}
{0, 1}


In [25]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [26]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [27]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
import os

os.environ['WANDB_DISABLED'] = 'true'

bertmodel = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none",
)

trainer = Trainer(
    model=bertmodel,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.680115,0.552500,0.532508,0.860000,0.657744
2,No log,0.667771,0.662500,0.652582,0.695000,0.673123
3,No log,0.651371,0.690000,0.658333,0.790000,0.718182


Evaluation results: {'eval_loss': 0.6513712406158447, 'eval_accuracy': 0.69, 'eval_precision': 0.6583333333333333, 'eval_recall': 0.79, 'eval_f1': 0.7181818181818183, 'eval_runtime': 149.5283, 'eval_samples_per_second': 2.675, 'eval_steps_per_second': 0.047, 'epoch': 3.0}


In [28]:
# Save the best model
model_save_path = '/kaggle/working/best_bert_model.'
trainer.save_model(model_save_path)


In [31]:
import torch

# Save the model as a .pkl file
model_save_path = '/kaggle/working/bert_model.pkl'
torch.save(bertmodel.state_dict(), model_save_path)
